In [28]:
import bs4
import requests
import urllib.request
from selenium import webdriver
from urllib import parse
import time
import re
from selenium.webdriver.common.keys import Keys
import pandas as pd

#장르별 리스트,독점리스트 따로구성
def munpia_title_list():

    # 중복 제거
    def duplicate(my_list):
        my_set = set(my_list) # 집합으로 변환
        new_list = list(my_set) # 리스트로 변환
        return new_list



    driver = webdriver.Chrome("C:/chromedriver.exe")
    url="https://mm.munpia.com/?menu=member&action=login"
    driver.get(url)
    
    time.sleep(2)
    elem_login = driver.find_element_by_xpath('//*[@id="member_id"]') # 현재 크롬에 떠 있는 웹페이지에서 id 속성 값이 id 인 element를 찾음
    elem_login.clear()
    elem_login.send_keys("narukane") # typing

    elem_login = driver.find_element_by_xpath('//*[@id="member_pw"]')
    elem_login.clear()
    elem_login.send_keys("ksj3721*")
    
    elem_enter=driver.find_element_by_xpath('//*[@id="sub_contents"]/article/div/div[1]/div[4]/form/table/tbody/tr[4]/td/input')
    elem_enter.click()


    url = "https://mm.munpia.com/?menu=novel&action=list&section=pl.all"
    #url = "https://mm.munpia.com/main/v42/slide/pl/tab/plserial/subtab/serial_end/selectbox//s"
    #문피아 완결 : https://mm.munpia.com/main/v42/slide/pl/tab/plserial/subtab/serial_end/selectbox//s/
    #성인 https://mm.munpia.com/?menu=novel&action=list&section=pl.all
    driver.get(url)
    bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")

    SCROLL_PAUSE_TIME = 2.0
    title_list = []

    n=1
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    base_xpath='//*[@id="list-pl"]/ul/li[{}]/a/div/div[2]/p[3]'
    while True:
        while True:
            xpath=base_xpath.format(n)
            try:
                box_area=driver.find_element_by_xpath(xpath)
            except:
                n=n+1
                break
            title_name=box_area.text.replace(u'\u4ede', '')
            title_list.append(title_name)
            n=n+1

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(1)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_UP)
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_UP)
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(SCROLL_PAUSE_TIME)
        if new_height == last_height:
            break
        last_height = new_height
      

    list_num = len(title_list) # 검색 결과 개수

    for i in range(0, list_num):
        print("제목 : " + title_list[i] )

    # csv 파일로 저장
    data = pd.DataFrame(title_list, columns = ['제목'])
    data.to_csv('munpia_title_adult_list.csv', encoding='cp949', index=False)

In [29]:
munpia_title_list()